## Agriculture CDC Data 2019
Data source: https://data.world/cdc2019/agriculture

### Imports and Initialization

In [1]:
import pandas as pd

### Functions

In [2]:
# print all rows
def display_all(df):
    pd.set_option('display.max_rows', None)
    print(df)
    pd.reset_option('display.max_rows')

In [3]:
# Given commodity return specific attributes related to the commodity
def get_attributes(commodity):
    return pd.DataFrame(columns = df_main[df_main['Commodity'] == commodity]['Attribute'].unique())

In [4]:
# Given attribute name, index and 2 dataframe will find value based on attribute
def get_cell (attribute, index, df_main, df_transpose):
        
        # iterate main data frame to look for values based on the attribute
        for index_main in range(0, len(df_main)):
                
            condition = ((df_main.loc[index_main, 'Attribute'] == attribute) & 
                         (df_main.loc[index_main, 'Commodity'] == df_transpose.loc[index, 'Commodity']) & 
                         (df_main.loc[index_main, 'Year'] == df_transpose.loc[index, 'Year'])) 

            if (condition):
                return [df_main.loc[index_main,'Value'] , df_main.loc[index_main,'Units']]
            else:
                continue
        return (-1)

In [5]:
# Transpose the main data frame for more manageable analysis
def tranpose_table (df_main, df_agriculture, df_units):
    
    # Get first commodity and attributes
    prev_commodity = df_agriculture.iloc[0]['Commodity']
    attributes = get_attributes(prev_commodity)
    
    for index in range(0, len(df_agriculture)):

        # only get new attributes if this is a different commodity from previous iteration
        cur_commodity = df_agriculture.iloc[index]['Commodity']

        if (prev_commodity != cur_commodity):
            prev_commodity = cur_commodity
            attributes = get_attributes(prev_commodity)

        # Iterate all attributes
        for attribute in attributes:

            # replace value of attribute with values from main data frame
            cells = get_cell(attribute, index, df_main, df_agriculture)

            if ((cells[0] != (-1)) & (cells[1] != (-1))):
                df_agriculture.loc[index, attribute] = cells[0]
                df_units.loc[index, attribute] = cells[1]         
    
    # # Fill Nan with zero values
    # df_agriculture.fillna(0, inplace = True)
    # df_units.fillna(0, inplace = True)
    
    return (0)

In [6]:
# Save data frame in to csv
def save_data_frame_to_csv(df, title):
    df.to_csv(title)

### Load CSV File into DataFrame

In [7]:
# main table from CSV file
df_main = pd.DataFrame(pd.read_csv('dataset/Projection2019.csv'))
df_main.reset_index()

,index,Attribute,Commodity,CommodityType,Units,YearType,Year,Value
0,0,Beginning stocks,Barley,Crops,Million bushels,Market year,2017/18,106.0
1,1,Beginning stocks,Barley,Crops,Million bushels,Market year,2018/19,94.0
2,2,Beginning stocks,Barley,Crops,Million bushels,Market year,2019/20,88.0
3,3,Beginning stocks,Barley,Crops,Million bushels,Market year,2020/21,87.0
4,4,Beginning stocks,Barley,Crops,Million bushels,Market year,2021/22,83.0
...,...,...,...,...,...,...,...,...
2923,2923,Yield/harvested acre,Wheat,Crops,Bushels per acre,Market year,2024/25,49.8
2924,2924,Yield/harvested acre,Wheat,Crops,Bushels per acre,Market year,2025/26,50.2
2925,2925,Yield/harvested acre,Wheat,Crops,Bushels per acre,Market year,2026/27,50.6
2926,2926,Yield/harvested acre,Wheat,Crops,Bushels per acre,Market year,2027/28,51.0


### Transpose Loaded Data into a new Data Frame

In [8]:
# create a new table
df_agriculture = pd.DataFrame(pd.read_csv('dataset/Projection2019.csv'))
df_units = pd.DataFrame(pd.read_csv('dataset/Projection2019.csv'))

# Get all attributes from the main table 
attributes = pd.DataFrame(columns = df_main['Attribute'].unique())

# Concatenate the columns to the new table
df_agriculture = pd.concat([df_agriculture, attributes], axis = 1)
df_units = pd.concat([df_units, attributes], axis = 1)

# Set new index and drop 'Attribute', 'Units' and 'Value' columns
drop_columns = ['Attribute', 'Value', 'Units']
df_agriculture.drop(columns = drop_columns, inplace = True)
df_units.drop(columns = drop_columns, inplace = True)

# Drop duplicates
df_agriculture.drop_duplicates(inplace = True)
df_units.drop_duplicates(inplace = True)

# Reset table index before iterating to fill in NaN values
df_agriculture.reset_index(drop = True, inplace = True)
df_units.reset_index(drop = True, inplace = True)

# Transpose df_main into df_agriculture and df_units
tranpose_table(df_main, df_agriculture, df_units)

# Save to CSV
df_agriculture.to_csv('dataset/transposed_agriculture.csv')
df_units.to_csv('dataset/transpose_units.csv')

### Tranposed tables

In [9]:
# Agriculture table tranposed
df_agriculture

,Commodity,CommodityType,YearType,Year,Beginning stocks,Domestic,Ending stocks,Exports,Farm price,Feed & residual,...,Total demand,"Soybean oil, for methyl ester",Crush,Crush margin,Seed & residual,"Hen turkeys, National","Turkey, farm",Variable cost of production (per pound),Food,Seed
0,Barley,Crops,Market year,2017/18,106.0,158.0,94.0,5.0,4.47,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Barley,Crops,Market year,2018/19,94.0,170.0,88.0,5.0,4.65,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Barley,Crops,Market year,2019/20,88.0,175.0,87.0,5.0,4.75,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Barley,Crops,Market year,2020/21,87.0,180.0,83.0,5.0,4.75,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Barley,Crops,Market year,2021/22,83.0,185.0,83.0,5.0,4.8,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Wheat,Crops,Market year,2024/25,942.0,1180.0,959.0,1000.0,5.2,120.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,995.0,65.0
200,Wheat,Crops,Market year,2025/26,959.0,1184.0,953.0,1000.0,5.1,120.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999.0,65.0
201,Wheat,Crops,Market year,2026/27,953.0,1188.0,960.0,1000.0,5.0,120.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1003.0,65.0
202,Wheat,Crops,Market year,2027/28,960.0,1192.0,959.0,1000.0,5.0,120.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1007.0,65.0


#### Tranposed Units table

In [10]:
# Units table tranposed
pd.set_option('display.max_columns', None)
df_units

,Commodity,CommodityType,YearType,Year,Beginning stocks,Domestic,Ending stocks,Exports,Farm price,Feed & residual,"Food, seed, & industrial",Harvested acres,Imports,Loan rate,Planted acres,Production,Returns over variable costs (net returns),"Stocks/use ratio, percent",Supply total,Total use/disposition,Variable cost of production (per acre),Variable cost of production (per bushel),Yield/harvested acre,"Beef cattle, farm",Beef cow inventory,"Calves, farm",Cattle inventory,Commercial production,"Feeder steers, Okla. City","Production, farm","Steers, 5-area, all grades",Total cow inventory,Total disappearance,Total production,"Broilers, farm",Disappearance,Federally inspected slaughter,National composite price,Ethanol for fuel,Commercial use: Milkfat basis,Commercial use: Skim solids,Milk per cow,Milk production,Number of cows,Prices: All milk,"Eggs, farm",Hatching use,"New York, Grade A large",Per capita,Retail weight: Broilers,Retail weight: Lamb & mutton,Retail weight: Other chicken,Retail weight: Red meat & poultry,Retail weight: Total beef,Retail weight: Total pork,Retail weight: Total poultry,Retail weight: Total red meat,Retail weight: Total veal,Retail weight: Turkeys,"Hog, inventory Dec.1, previous year","Hogs, farm","National base, live eqv","Milling rate, percent",Variable cost of production (per cwt.),Crushing yields,Prices,Total demand,"Soybean oil, for methyl ester",Crush,Crush margin,Seed & residual,"Hen turkeys, National","Turkey, farm",Variable cost of production (per pound),Food,Seed
0,Barley,Crops,Market year,2017/18,Million bushels,Million bushels,Million bushels,Million bushels,Dollars per bushel,Million bushels,Million bushels,Million acres,Million bushels,Dollars per bushel,Million acres,Million bushels,Dollars per acre,Percent,Million bushels,Million bushels,Dollars,Dollars,Bushels per acre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Barley,Crops,Market year,2018/19,Million bushels,Million bushels,Million bushels,Million bushels,Dollars per bushel,Million bushels,Million bushels,Million acres,Million bushels,Dollars per bushel,Million acres,Million bushels,Dollars per acre,Percent,Million bushels,Million bushels,Dollars,Dollars,Bushels per acre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Barley,Crops,Market year,2019/20,Million bushels,Million bushels,Million bushels,Million bushels,Dollars per bushel,Million bushels,Million bushels,Million acres,Million bushels,Dollars per bushel,Million acres,Million bushels,Dollars per acre,Percent,Million bushels,Million bushels,Dollars,Dollars,Bushels per acre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Barley,Crops,Market year,2020/21,Million bushels,Million bushels,Million bushels,Million bushels,Dollars per bushel,Million bushels,Million bushels,Million acres,Million bushels,Dollars per bushel,Million acres,Million bushels,Dollars per acre,Percent,Million bushels,Million bushels,Dollars,Dollars,Bushels per acre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Barley,Crops,Market year,2021/22,Million bushels,Million bushels,Million bushels,Million bushels,Dollars per bushel,Million bushels,Million bushels,Million acres,Million bushels,Dollars per bushel,Million acres,Million bushels,Dollars per acre,Percent,Million bushels,Million bushels,Dollars,Dollars,Bushels per acre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,